In [ ]:
#import data
import pandas as pd

# 엑셀 파일 불러오기
excel_path1 = '/Users/kyuree/Desktop/pds_capstone1/dataset/accidentInfoList.xlsx'
xls1 = pd.ExcelFile(excel_path1)

# 각 시트를 데이터프레임으로 읽어와 리스트에 저장
dfs1 = [pd.read_excel(xls1, sheet_name=sheet_name) for sheet_name in xls1.sheet_names]

# 시트별 데이터프레임을 행 기준으로 통합
accident_origin = pd.concat(dfs1, axis=0, ignore_index=True)

# 결과 출력
accident_origin

In [ ]:
#1. 필요한 컬럼만 필터링 
''' 
'사고일시', '시군구', '사고내용', '사망자수', '중상자수', '경상자수', '부상신고자수', '사고유형','법규위반','도로형태', '가해운전자 상해정도','피해운전자 상해정도'
'''
filtered_accident = accident_origin[['사고일시', '시군구', '사고내용', '사망자수', '중상자수', '경상자수', '부상신고자수', '사고유형','법규위반','도로형태', '가해운전자 상해정도','피해운전자 상해정도']]
filtered_accident

In [ ]:
#사고일시 -> 사고발생년도 / 월 / 일 / 시간으로 각각 구분하는 컬럼으로 재구성

filtered_accident['사고일시'] = filtered_accident['사고일시'].str.replace(' ', '') # 띄어쓰기 제거

for i in range(len(filtered_accident)):
    if len(filtered_accident.loc[i,'사고일시']) ==12:
        filtered_accident.loc[i,'사고발생_년도'] = filtered_accident.loc[i, '사고일시'][:5]
        filtered_accident.loc[i,'사고발생_월일'] = filtered_accident.loc[i, '사고일시'][5:9]
        filtered_accident.loc[i,'사고발생_시간'] = filtered_accident.loc[i, '사고일시'][9:]
    elif len(filtered_accident.loc[i,'사고일시']) ==13:
        filtered_accident.loc[i,'사고발생_년도'] = filtered_accident.loc[i, '사고일시'][:5]
        filtered_accident.loc[i,'사고발생_월일'] = filtered_accident.loc[i, '사고일시'][5:10]
        filtered_accident.loc[i,'사고발생_시간'] = filtered_accident.loc[i, '사고일시'][10:]
    else:
        filtered_accident.loc[i,'사고발생_년도'] = filtered_accident.loc[i, '사고일시'][:5]
        filtered_accident.loc[i,'사고발생_월일'] = filtered_accident.loc[i, '사고일시'][5:11]
        filtered_accident.loc[i,'사고발생_시간'] = filtered_accident.loc[i, '사고일시'][11:]
filtered_accident

In [ ]:
#확인
print(filtered_accident['사고발생_년도'].value_counts())
print(filtered_accident['사고발생_월일'].value_counts())
print(filtered_accident['사고발생_시간'].value_counts())

In [ ]:
#시군구 -> 서울특별시 노원구 / **동 으로 구분하는 컬럼 생성
filtered_accident['시군구'].value_counts()

for i in range(len(filtered_accident)):
    filtered_accident.loc[i,'행정동'] = filtered_accident.loc[i,'시군구'][10:]
filtered_accident[['시군구','행정동']]

In [ ]:
#사고유형 -> 차대사람 / 세부사고유형 -> 횡단중으로 컬럼 분리
filtered_accident['사고유형'].value_counts()

for i in range(len(filtered_accident)):
    filtered_accident.loc[i,'사고세부유형'] = filtered_accident.loc[i,'사고유형'][7:]
filtered_accident[['사고유형','사고세부유형']]

In [ ]:
#널값 확인
print(filtered_accident.isnull().sum())
print(filtered_accident['피해운전자 상해정도'].value_counts())

In [ ]:
#도로형태 -> 도로형태(단일로) / 세부도로형태(기타)로 분리, 큰도로형태에 도로형태 넣음
for i in range(len(filtered_accident)):
    if filtered_accident.loc[i,'도로형태'][:2]=='기타':
        filtered_accident.loc[i,'세부도로형태'] = filtered_accident.loc[i,'도로형태'][5:]
        filtered_accident.loc[i,'큰도로형태'] = filtered_accident.loc[i,'도로형태'][:2]
    # elif len(filtered_accident.loc[i,'도로형태'][:3]) ==3:
    else:
        filtered_accident.loc[i,'세부도로형태'] = filtered_accident.loc[i,'도로형태'][6:]
        filtered_accident.loc[i,'큰도로형태'] = filtered_accident.loc[i,'도로형태'][:3]
filtered_accident[['도로형태','큰도로형태','세부도로형태']]

In [ ]:
#최종 전처리된 교통사고 데이터
res_accident = filtered_accident[['사고발생_년도','사고발생_월일','사고발생_시간','행정동','사고내용','사망자수','중상자수', '경상자수', '부상신고자수','사고세부유형', '법규위반',
       '큰도로형태','세부도로형태', '가해운전자 상해정도', '피해운전자 상해정도']]
res_accident

In [ ]:
# 데이터프레임을 CSV 파일로 저장
res_accident.to_csv('kr_교통사고데이터_전처리.csv', index=False)